In [ ]:
import ccxt
import requests
import json

In [ ]:
with open("secret.json", "r") as f:
    api_key = json.load(f)['devtest']

authentification = {
    "apiKey": api_key['apiKey'],
    "secret": api_key['secret'],
    "password": api_key['password'],
}

bitget = ccxt.bitget(authentification)

In [ ]:
balance = bitget.fetch_balance(params={'defaultType': 'future'})['USDT']
print(balance)

In [ ]:
def place_order(symbol, side, amount, reduce_only=False):
    amount = bitget.amount_to_precision(symbol, amount)
    order = bitget.create_order(symbol, 'market', side, amount, params={'marginMode': 'isolated', 'reduceOnly': reduce_only})
    order_info = bitget.fetch_order(order['id'], symbol) # https://www.bitget.com/api-doc/contract/trade/Get-Order-Details
    print(order_info)


In [ ]:
symbol = 'LINK/USDT:USDT'
side = 'buy'
amount = 3

place_order(symbol, side, amount)

In [ ]:
symbol = 'LINK/USDT:USDT'
side = 'sell'
amount = 3

place_order(symbol, side, amount, reduce_only=True)

In [ ]:
def send_order_info_to_discord(order_info, webhook_url):
    balance = bitget.fetch_balance(params={'defaultType': 'future'})['USDT']
    title = symbol
    description = (
        f"Amount: {order_info['info']['size']}\n"
        f"Average Price: {order_info['info']['priceAvg']}\n"
        f"Total PnL: {order_info['info']['totalProfits']}\n"
        f"Fee: {order_info['info']['fee']}\n"
        f"Free account balance : {round(balance['free'], 2)}\n"
        f"Total account balance : {round(balance['total'], 2)}\n"
        
    )
    if order_info['side'] == "buy":
        color = 0x00FF00  # Green
    elif order_info['side'] == "sell":
        color = 0xFF0000  # Red

    data = {
        "embeds": [{
            "title": title,
            "description": description,
            "color": color
        }]
    }

    requests.post(webhook_url, json=data)

In [ ]:
def place_order_and_send_discord_notif(symbol, side, amount, reduce_only=False):
    webhook_url = 'https://discord.com/api/webhooks/1274701683816927349/CgWWp8Wu-JTRU1nj2aLq8Rcba5EpURU5E-dd4hVK_3eqOrvVqi-iGayqZQQV0_mfqfCL'
    amount = bitget.amount_to_precision(symbol, amount)
    order = bitget.create_order(symbol, 'market', side, amount, params={'marginMode': 'isolated', 'reduceOnly': reduce_only})
    order_info = bitget.fetch_order(order['id'], symbol)
    send_order_info_to_discord(order_info, webhook_url)

In [ ]:
symbol = 'LINK/USDT:USDT'
side = 'buy'
amount = 2

place_order_and_send_discord_notif(symbol, side, amount)

In [ ]:
symbol = 'LINK/USDT:USDT'
side = 'sell'
amount = 2

place_order_and_send_discord_notif(symbol, side, amount, reduce_only=True)